# Requirements
1. Before running this notebook, make sure you have 
```
DRCD_plus_FGC_training.json
FGC_all_mocks_DRCD_format.json
FGC_final_DRCD_format.json
```
in the current directory.
2. When **not running on Google Colab**, comment out `pip install transformers`
3. If you have `tensorboard` jupyter notebook plugin, use `%load_ext tensorboard`

In [1]:
# !pip install transformers
%load_ext tensorboard
import glob
import logging
import os
import random
import argparse
import json
import timeit
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from transformers import (
  BertTokenizer,
  AlbertForQuestionAnswering,
  AutoConfig,
  AdamW,
  get_linear_schedule_with_warmup,
  squad_convert_examples_to_features,
)

from transformers.data.metrics.squad_metrics import (
  compute_predictions_log_probs,
  compute_predictions_logits,
  squad_evaluate,
)

from transformers.data.processors.squad import SquadResult, SquadProcessor
try:
  from torch.utils.tensorboard import SummaryWriter
except ImportError:
  from tensorboardX import SummaryWriter

In [ ]:
pretrained = 'voidful/albert_chinese_base'
tokenizer = BertTokenizer.from_pretrained(pretrained)
model = AlbertForQuestionAnswering.from_pretrained(pretrained)
config = AutoConfig.from_pretrained(pretrained)

In [12]:
class Args(object):
  model_type = "ALBERT"
  model_name_or_path = "."
  output_dir = "output_training"
  data_dir = "."
  null_score_diff_threshold = 0.0 # If null_score - best_non_null is greater than the threshold predict null
  version_2_with_negative = False
  max_seq_length = 512
  doc_stride = 128
  max_query_length = 128
  do_train = True
  do_eval = True
  evaluate_during_training = True
  eval_all_checkpoints = True
  do_lower_case = True
  per_gpu_train_batch_size = 8
  per_gpu_eval_batch_size = 4
  no_cuda = False

  learning_rate = 3e-5
  gradient_accumulation_steps = 8
  weight_decay = 0.0
  adam_epsilon = 1e-8
  max_grad_norm = 1.
  num_train_epochs = 3.
  max_steps = -1 # If > 0: set total number of training steps to perform. Override num_train_epochs.
  warmup_steps = 200
  n_best_size = 5
  max_answer_length = 200
  logging_steps = 1000
  save_steps = 1000

  seed = 1337
  local_rank = -1
  threads = 4
  n_gpu = 1
  overwrite_cache = False
  overwrite_output_dir = True
  verbose_logging = False

In [4]:
class DRCDProcessor(SquadProcessor):
  train_file = "./DRCD_plus_FGC_training.json"
  dev_file = "./FGC_all_mocks_DRCD_format.json"

def set_seed(args):
  random.seed(args.seed)
  np.random.seed(args.seed)
  torch.manual_seed(args.seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def to_list(tensor):
  return tensor.detach().cpu().tolist()

In [5]:
def train(args, train_dataset, model, tokenizer, config):
  """ Train the model """
  tb_writer = SummaryWriter()
  logger = logging.getLogger(__name__)

  args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
  train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

  if args.max_steps > 0:
    t_total = args.max_steps
    args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
  else:
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

  # Prepare optimizer and schedule (linear warmup and decay)
  no_decay = ["bias", "LayerNorm.weight"]
  optimizer_grouped_parameters = [
    {
      "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
  )

  # Train!
  logger.info("***** Running training *****")
  logger.info("  Num examples = %d", len(train_dataset))
  logger.info("  Num Epochs = %d", args.num_train_epochs)
  logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
  logger.info(
    "  Total train batch size (w. parallel, distributed & accumulation) = %d",
    args.train_batch_size
    * args.gradient_accumulation_steps
    * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
  )
  logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
  logger.info("  Total optimization steps = %d", t_total)

  global_step = 1
  epochs_trained = 0
  steps_trained_in_current_epoch = 0

  logger.info("  Starting fine-tuning.")

  tr_loss, logging_loss = 0.0, 0.0

  model.zero_grad()
  train_iterator = trange(
    epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0], position=0, leave=False
  )
  # Added here for reproductibility
  set_seed(args)

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0], position=1, leave=False)
    for step, batch in enumerate(epoch_iterator):

      # Skip past any already trained steps if resuming training
      if steps_trained_in_current_epoch > 0:
        steps_trained_in_current_epoch -= 1
        continue

      model.train()
      batch = tuple(t.to(args.device) for t in batch)

      inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
        "start_positions": batch[3],
        "end_positions": batch[4],
      }

      if args.model_type in ["xlm", "roberta", "distilbert", "camembert"]:
        del inputs["token_type_ids"]

      outputs = model(**inputs)
      # model outputs are always tuple in transformers (see doc)
      loss = outputs[0]

      if args.n_gpu > 1:
        loss = loss.mean()  # mean() to average on multi-gpu parallel (not distributed) training
      if args.gradient_accumulation_steps > 1:
        loss = loss / args.gradient_accumulation_steps


      loss.backward()

      tr_loss += loss.item()
      if (step + 1) % args.gradient_accumulation_steps == 0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        global_step += 1

        # Log metrics
        if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
          # Only evaluate when single GPU otherwise metrics may not average well
          if args.local_rank == -1 and args.evaluate_during_training:
            results = evaluate(args, model, tokenizer, prefix="step_{}".format(global_step))
            for key, value in results.items():
              tb_writer.add_scalar("eval_{}".format(key), value, global_step)
          tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
          tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
          logging_loss = tr_loss

        # Save model checkpoint
        if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
          output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
          # Take care of distributed/parallel training
          model_to_save = model.module if hasattr(model, "module") else model
          model_to_save.save_pretrained(output_dir)
          tokenizer.save_pretrained(output_dir)

          torch.save(args, os.path.join(output_dir, "training_args.bin"))
          logger.info("Saving model checkpoint to %s", output_dir)

          torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
          torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
          logger.info("Saving optimizer and scheduler states to %s", output_dir)

      if args.max_steps > 0 and global_step > args.max_steps:
        epoch_iterator.close()
        break
    if args.max_steps > 0 and global_step > args.max_steps:
      train_iterator.close()
      break

  if args.local_rank in [-1, 0]:
    tb_writer.close()

  return global_step, tr_loss / global_step


def evaluate(args, model, tokenizer, prefix=""):
  logger = logging.getLogger(__name__)
  dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)

  if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
    os.makedirs(args.output_dir)

  args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)

  # Note that DistributedSampler samples randomly
  eval_sampler = SequentialSampler(dataset)
  eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

  # multi-gpu evaluate
  if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
    model = torch.nn.DataParallel(model)

  # Eval!
  logger.info("***** Running evaluation {} *****".format(prefix))
  logger.info("  Num examples = %d", len(dataset))
  logger.info("  Batch size = %d", args.eval_batch_size)

  all_results = []
  start_time = timeit.default_timer()

  for batch in tqdm(eval_dataloader, desc="Evaluating", position=0, leave=False):
    model.eval()
    batch = tuple(t.to(args.device) for t in batch)

    with torch.no_grad():
      inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
      }

      if args.model_type in ["xlm", "roberta", "distilbert", "camembert"]:
        del inputs["token_type_ids"]

      feature_indices = batch[3]

      outputs = model(**inputs)

    for i, feature_index in enumerate(feature_indices):
      eval_feature = features[feature_index.item()]
      unique_id = int(eval_feature.unique_id)

      output = [to_list(output[i]) for output in outputs]

      # Some models (XLNet, XLM) use 5 arguments for their predictions, while the other "simpler"
      # models only use two.
      if len(output) >= 5:
        start_logits = output[0]
        start_top_index = output[1]
        end_logits = output[2]
        end_top_index = output[3]
        cls_logits = output[4]

        result = SquadResult(
          unique_id,
          start_logits,
          end_logits,
          start_top_index=start_top_index,
          end_top_index=end_top_index,
          cls_logits=cls_logits,
        )

      else:
        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)

      all_results.append(result)

  evalTime = timeit.default_timer() - start_time
  logger.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

  # Compute predictions
  output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
  output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
  output_null_log_odds_file = None


  predictions = compute_predictions_logits(
    examples,
    features,
    all_results,
    args.n_best_size,
    args.max_answer_length,
    args.do_lower_case,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    args.verbose_logging,
    args.version_2_with_negative,
    args.null_score_diff_threshold,
    tokenizer,
  )

  # Compute the F1 and exact scores.
  results = squad_evaluate(examples, predictions)
  return results


def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False, processor=DRCDProcessor()):
  logger = logging.getLogger(__name__)
  # Load data features from cache or dataset file
  input_dir = args.data_dir if args.data_dir else "."
  cached_features_file = os.path.join(
    input_dir,
    "cached_{}_{}_{}".format(
      "dev" if evaluate else "train",
      list(filter(None, (processor.dev_file if evaluate else processor.train_file).split('/'))).pop(),
      str(args.max_seq_length),
    ),
  )

  # Init features and dataset from cache if it exists
  if os.path.exists(cached_features_file) and not args.overwrite_cache:
    logger.info("Loading features from cached file %s", cached_features_file)
    features_and_dataset = torch.load(cached_features_file)
    features, dataset, examples = (
      features_and_dataset["features"],
      features_and_dataset["dataset"],
      features_and_dataset["examples"],
    )
  else:
    logger.info("Creating features from dataset file at %s", input_dir)

    if evaluate:
      examples = processor.get_dev_examples(args.data_dir)
    else:
      examples = processor.get_train_examples(args.data_dir)

    features, dataset = squad_convert_examples_to_features(
      examples=examples,
      tokenizer=tokenizer,
      max_seq_length=args.max_seq_length,
      doc_stride=args.doc_stride,
      max_query_length=args.max_query_length,
      is_training=not evaluate,
      return_dataset="pt",
      threads=args.threads,
    )

    if args.local_rank in [-1, 0]:
      logger.info("Saving features into cached file %s", cached_features_file)
      torch.save({"features": features, "dataset": dataset, "examples": examples}, cached_features_file)

  if args.local_rank == 0 and not evaluate:
    # Make sure only the first process in distributed training process the dataset, and the others will use the cache
    torch.distributed.barrier()

  if output_examples:
    return dataset, examples, features
  return dataset

In [6]:
def main(args, model, tokenizer, config):
  logger = logging.getLogger(__name__)
  if args.doc_stride >= args.max_seq_length - args.max_query_length:
    logger.warning(
      "WARNING - You've set a doc stride which may be superior to the document length in some "
      "examples. This could result in errors when building features from the examples. Please reduce the doc "
      "stride or increase the maximum length to ensure the features are correctly built."
    )

  if (
    os.path.exists(args.output_dir)
    and os.listdir(args.output_dir)
    and args.do_train
    and not args.overwrite_output_dir
  ):
    raise ValueError(
      "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
        args.output_dir
      )
    )

  # Setup CUDA, GPU & distributed training
  if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
  else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    args.n_gpu = 1
  args.device = device

  # Setup logging
  logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
  )
  logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s",
    args.local_rank,
    device,
    args.n_gpu,
    bool(args.local_rank != -1),
  )

  # Set seed
  set_seed(args)
  args.model_type = args.model_type.lower()

  model.to(args.device)

  # Training
  if args.do_train:
    train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)
    global_step, tr_loss = train(args, train_dataset, model, tokenizer, config)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

  # Save the trained model and the tokenizer
  if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
    logger.info("Saving model checkpoint to %s", args.output_dir)
    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    # Take care of distributed/parallel training
    model_to_save = model.module if hasattr(model, "module") else model
    model_to_save.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)

    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

  # Evaluation - we can ask to evaluate all the checkpoints (sub-directories) in a directory
  results = {}
  if args.do_eval and args.local_rank in [-1, 0]:
    if args.do_train:
      logger.info("Loading checkpoints saved during training for evaluation")
      checkpoints = [args.output_dir]
      if args.eval_all_checkpoints:
        checkpoints = list(
          os.path.dirname(c)
          for c in sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
        )
        logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce model loading logs
    else:
      logger.info("Loading checkpoint %s for evaluation", args.model_name_or_path)
      checkpoints = [args.model_name_or_path]

    logger.info("Evaluate the following checkpoints: %s", checkpoints)

    for checkpoint in checkpoints:
      # Reload the model
      global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
      model = AlbertForQuestionAnswering.from_pretrained(checkpoint)
      model.to(args.device)

      # Evaluate
      result = evaluate(args, model, tokenizer, prefix=global_step)

      result = dict((k + ("_{}".format(global_step) if global_step else ""), v) for k, v in result.items())
      results.update(result)

  logger.info("Results: {}".format(results))

  return results


In [ ]:
main(Args(), model, tokenizer, config)

In [ ]:
class TestSetProcessor(DRCDProcessor):
  dev_file = "./FGC_final_DRCD_format.json"

test_args = Args()
test_args.do_train=False
device = torch.device("cuda" if torch.cuda.is_available() and not test_args.no_cuda else "cpu")
test_args.n_gpu = 0 if test_args.no_cuda else torch.cuda.device_count()
test_args.device = device

def evaluate_test_set(args, model, tokenizer, prefix=""):
  logger = logging.getLogger(__name__)
  dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True, \
                                                        processor=TestSetProcessor())

  if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
    os.makedirs(args.output_dir)

  args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)

  # Note that DistributedSampler samples randomly
  eval_sampler = SequentialSampler(dataset)
  eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

  # multi-gpu evaluate
  if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
    model = torch.nn.DataParallel(model)

  # Eval!
  logger.info("***** Running evaluation {} *****".format(prefix))
  logger.info("  Num examples = %d", len(dataset))
  logger.info("  Batch size = %d", args.eval_batch_size)

  all_results = []
  start_time = timeit.default_timer()

  for batch in tqdm(eval_dataloader, desc="Evaluating", position=0, leave=False):
    model.eval()
    batch = tuple(t.to(args.device) for t in batch)

    with torch.no_grad():
      inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
      }

      if args.model_type in ["xlm", "roberta", "distilbert", "camembert"]:
        del inputs["token_type_ids"]

      feature_indices = batch[3]

      outputs = model(**inputs)

    for i, feature_index in enumerate(feature_indices):
      eval_feature = features[feature_index.item()]
      unique_id = int(eval_feature.unique_id)

      output = [to_list(output[i]) for output in outputs]

      # Some models (XLNet, XLM) use 5 arguments for their predictions, while the other "simpler"
      # models only use two.
      if len(output) >= 5:
        start_logits = output[0]
        start_top_index = output[1]
        end_logits = output[2]
        end_top_index = output[3]
        cls_logits = output[4]

        result = SquadResult(
          unique_id,
          start_logits,
          end_logits,
          start_top_index=start_top_index,
          end_top_index=end_top_index,
          cls_logits=cls_logits,
        )

      else:
        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)

      all_results.append(result)

  evalTime = timeit.default_timer() - start_time
  logger.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

  # Compute predictions
  output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
  output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
  output_null_log_odds_file = None


  predictions = compute_predictions_logits(
    examples,
    features,
    all_results,
    args.n_best_size,
    args.max_answer_length,
    args.do_lower_case,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    args.verbose_logging,
    args.version_2_with_negative,
    args.null_score_diff_threshold,
    tokenizer,
  )

  # Compute the F1 and exact scores.
  results = squad_evaluate(examples, predictions)
  return results

test_results = {}
checkpoints = list(
          os.path.dirname(c)
          for c in sorted(glob.glob(test_args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
        )
logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce model loading logs
logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce model loading logs
logger = logging.getLogger(__name__)
logger.info("Evaluate the following checkpoints: %s", checkpoints)

for checkpoint in checkpoints:
  # Reload the model
  global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
  model = AlbertForQuestionAnswering.from_pretrained(checkpoint)
  model.to(test_args.device)

  # Evaluate
  result = evaluate_test_set(test_args, model, tokenizer, prefix="test_set_" + str(global_step))

  result = dict((k + ("_{}".format(global_step) if global_step else ""), v) for k, v in result.items())
  test_results.update(result)


In [15]:
for k in test_results.keys():
    if k.startswith("exact_"):
        print("step {} EM {}".format(k.split('_')[-1], test_results[k]))

step 1000 EM 20.0
step training EM 20.0
